In [27]:
!pip install tabpfn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 7.1 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: n

In [58]:
import numpy as np
import pandas as pd

# 读取数据
train_df = pd.read_csv('/kaggle/input/szgh-dataset/船舶估值/train/train_df.csv')
train_shipinfo = pd.read_csv('/kaggle/input/szgh-dataset/船舶估值/train/train_shipinfo.csv')
test_df = pd.read_csv('/kaggle/input/szgh-dataset/船舶估值/test/test_df.csv')
test_shipinfo = pd.read_csv('/kaggle/input/szgh-dataset/船舶估值/test/test_shipinfo.csv')
price_index = pd.read_csv('/kaggle/input/szgh-dataset/船舶估值/train/price_index.csv')

# 拼接
train_df = pd.merge(train_df, train_shipinfo, how='left', on='shipno')
train_df = pd.merge(train_df, price_index, how='left', left_on='sale_date', right_on='date')
test_df = pd.merge(test_df, test_shipinfo, how='left', on='shipno')
test_df = pd.merge(test_df, price_index, how='left', left_on='sale_date', right_on='date')

# 合并训练集，测试集
train_df['is_test'] = 0
test_df['is_test'] = 1
df_all = pd.concat([train_df,test_df],axis=0,ignore_index=True)

In [59]:
# 基础时间特征
date_feature = ['sale_date', 'date']
df_all["sale_date"] = pd.to_datetime(df_all["sale_date"])
df_all["sale_date_year"] = df_all["sale_date"].dt.year
df_all["sale_date_month"] = df_all["sale_date"].dt.month
df_all['sale_date_year_sin'] = np.sin(2 * np.pi * df_all['sale_date_year'])
df_all['sale_date_month_sin'] = np.sin(2 * np.pi * df_all['sale_date_month'] / 12) 

# 季节性特征
# df_all['q1'] = df_all['sale_date_month'].apply(lambda x: 1 if x in [1,2,3] else 0)
# df_all['q2'] = df_all['sale_date_month'].apply(lambda x: 1 if x in [4,5,6] else 0)
# df_all['q3'] = df_all['sale_date_month'].apply(lambda x: 1 if x in [7,8,9] else 0)
# df_all['q4'] = df_all['sale_date_month'].apply(lambda x: 1 if x in [10,11,12] else 0)

In [60]:
# 船舶特征
df_all['ship_age'] = df_all['sale_date_year'] - df_all['built_date']
# df_all['ship_s'] = df_all['length'] - df_all['width']
# df_all['ship_v'] = df_all['length'] * df_all['width'] * df_all['width'] * df_all['height']
# df_all['ship_s/v'] = df_all['ship_s'] / df_all['ship_v']

# 组合特征
# df_all['comfactor+dryfactor'] = df_all['comfactor'] + df_all['dryfactor']
# df_all['comfactor-dryfactor'] = df_all['comfactor'] - df_all['dryfactor']
# df_all['comfactor*dryfactor'] = df_all['comfactor'] * df_all['dryfactor']
# df_all['comfactor/dryfactor'] = df_all['comfactor'] * df_all['dryfactor']

# 目标编码
# for col in ['gt','dwt','length','width']:
#     df_all[f'{col}_target_sum'] = df_all[col].map(df_all.groupby([col])['saleprice'].sum())
#     df_all[f'{col}_target_mean'] = df_all[col].map(df_all.groupby([col])['saleprice'].mean())
#     df_all[f'{col}_target_max'] = df_all[col].map(df_all.groupby([col])['saleprice'].max())
#     df_all[f'{col}_target_min'] = df_all[col].map(df_all.groupby([col])['saleprice'].min())
#     df_all[f'{col}_target_median'] = df_all[col].map(df_all.groupby([col])['saleprice'].median())

In [6]:
# # 市场指数变化特征
# df_all['market_3m_change'] = df_all.groupby('shipno')['dryfactor'].transform(lambda x: x.pct_change(3))
# df_all['market_6m_change'] = df_all.groupby('shipno')['dryfactor'].transform(lambda x: x.pct_change(6))

# # 市场相对位置特征
# df_all['market_position'] = pd.qcut(df_all['dryfactor'], q=5, labels=False)

In [61]:
# 类别编码
category_feature = df_all.select_dtypes(include=['object']).drop(columns=['shipno', 'date']).columns.tolist()
for c in category_feature:
    df_all[c],_ = df_all[c].factorize()
    df_all[c] -= df_all[c].min()
    df_all[c] = df_all[c].astype("int32")
    df_all[c] = df_all[c].astype("category")

In [62]:
# 分离
df_all = df_all.replace([-np.inf, np.inf],0)
train_df = df_all.iloc[:len(train_df)].reset_index(drop=True).copy()
test_df = df_all.iloc[len(train_df):].reset_index(drop=True).copy()

feats = [col for col in df_all if col not in ['shipno', 'sequence', 'saleprice', 'is_test'] + date_feature]                                                                                                                                                                                      
print('feats_num:',len(feats))

feats_num: 68


In [63]:
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor,LGBMClassifier
# from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score,mean_absolute_percentage_error


def lgb_mdoel(train, test, target, feats, seed):

    score = 0
    flods = 10
    kf = KFold(n_splits=flods, shuffle=True, random_state=seed)
    
    oof_lgb = np.zeros(len(train))
    pred_lgb = np.zeros(len(test))
    
    for i, (train_index, test_index) in enumerate(kf.split(train,train[target])):
    
        print(f"### Fold {i+1}")
        
        x_train = train.loc[train_index, feats].copy()
        y_train = train.loc[train_index, target]    
        x_valid = train.loc[test_index, feats].copy()
        y_valid = train.loc[test_index, target]
        x_test = test[feats].copy()
    
        model_lgb = LGBMRegressor(
            objective="regression", 
            boosting_type='gbdt',
            device="cpu", 
            max_depth=2, 
            colsample_bytree=0.3,  
            subsample=0.3, 
            n_estimators=2500, 
            learning_rate=0.01, 
            verbose=-1, 
            # early_stopping_rounds=100,
            random_seed=42,
        )
        model_lgb.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)],
        )
        
        pred_lgb_val = model_lgb.predict(x_valid)
        pred_lgb += model_lgb.predict(x_test)

        oof_lgb[test_index] = pred_lgb_val

        score_kold = mean_absolute_percentage_error(y_valid, pred_lgb_val)
        score += score_kold
        print('Score = ',score_kold) 
    
    pred_lgb /= flods
    score /= flods
    
    print(f"\nMean Score  = ", score)
    
    lgb_importance_df = pd.DataFrame({
        "Feature": feats,
        "Importance": model_lgb.feature_importances_ 
    }).sort_values(by="Importance", ascending=False)
    lgb_importance_df.head(10)
    
    return oof_lgb,pred_lgb,lgb_importance_df

In [64]:
seeds = [2025]
for seed in seeds:
    oof_lgb,pred_lgb,imp_lgb = lgb_mdoel(train_df, test_df, 'saleprice', feats, seed = seed)
    train_df[f'pred_seed_{seed}'] = oof_lgb
    test_df[f'pred_seed_{seed}'] = pred_lgb

### Fold 1
Score =  0.11557729879953373
### Fold 2
Score =  0.12802520402297266
### Fold 3
Score =  0.11968618170105098
### Fold 4
Score =  0.12030897750971538
### Fold 5
Score =  0.12049592099850572
### Fold 6
Score =  0.1344154479458823
### Fold 7
Score =  0.13980156081482942
### Fold 8
Score =  0.13777768647676447
### Fold 9
Score =  0.13588644236921973
### Fold 10
Score =  0.11843919357258767

Mean Score  =  0.1270413914211062


In [65]:
feats = [col for col in train_df if col not in ['shipno', 'sequence', 'saleprice', 'is_test'] + date_feature]                                                                                                                                                                                      
print('feats_num:',len(feats))

feats_num: 69


In [66]:
from tabpfn import TabPFNClassifier, TabPFNRegressor
import warnings
warnings.filterwarnings("ignore")

def nn_mdoel(train, test, target, feats, seed):

    score = 0
    flods = 10
    kf = KFold(n_splits=flods, shuffle=True, random_state=seed)
    
    oof = np.zeros(len(train))
    pred = np.zeros(len(test))

    
    for i, (train_index, test_index) in enumerate(kf.split(train,train[target])):
    
        print(f"### Fold {i+1}")
        
        x_train = train.loc[train_index, feats].copy()
        y_train = train.loc[train_index, target]    
        x_valid = train.loc[test_index, feats].copy()
        y_valid = train.loc[test_index, target]
        x_test = test[feats].copy()
    
        model = TabPFNRegressor()
        model.fit(
            x_train, y_train,
        )
        
        pred_val = model.predict(x_valid)
        pred += model.predict(x_test)

        oof[test_index] = pred_val

        score_kold = mean_absolute_percentage_error(y_valid, pred_val)
        score += score_kold
        print('Score = ',score_kold) 
    
    pred /= flods
    score /= flods
    
    print(f"\nMean Score  = ", score)
    
    return oof, pred

oof, pred = nn_mdoel(train_df, test_df, 'saleprice', feats, seed = 2025)

### Fold 1
Score =  0.07378849067558155
### Fold 2
Score =  0.0876130326763166
### Fold 3
Score =  0.07020969653292082
### Fold 4
Score =  0.07640854805529053
### Fold 5
Score =  0.084990377221241
### Fold 6
Score =  0.08253486590067965
### Fold 7
Score =  0.09335585176157886
### Fold 8
Score =  0.10428839319026799
### Fold 9
Score =  0.10835117266537793
### Fold 10
Score =  0.07702184493865451

Mean Score  =  0.08585622736179094


In [67]:
# 保存
pred_test = pred
submission = pd.DataFrame({'sequence':test_df['sequence'].astype('int'), 'saleprice':pred_test})
submission.to_csv('submission.csv',index=False)
submission.head(5)

,sequence,saleprice
0,0,7.769786
1,1,23.080729
2,2,21.964207
3,3,29.953551
4,4,35.559814
